# GPU内存

In [2]:
import torch

if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 查看释放后的显存状态
if torch.cuda.is_available():
    print(f"当前已分配显存: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"当前缓存中显存: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
else:
    print("No GPU ")

当前已分配显存: 0.00 GB
当前缓存中显存: 0.00 GB


# 1 准备

In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

print("正在加载 Embedding 模型...")
embed_model = HuggingFaceEmbedding(
    model_name="/root/nas-private/huggingface_cache/BAAI/bge-small-zh-v1.5"
)
# 应用到全局设置
Settings.embed_model = embed_model
print("Embedding 模型加载完成。")

HF_CACHE_DIR = "/root/nas-private/huggingface_cache"
LLM_RELATIVE_PATH = "Qwen/Qwen3-4B-instruct" 
LOCAL_LLM_PATH = os.path.join(HF_CACHE_DIR, LLM_RELATIVE_PATH)
model_name = LOCAL_LLM_PATH

正在加载 Embedding 模型...
Embedding 模型加载完成。


In [6]:
# Instruct 模型必须用这种格式包裹，否则它会把它当成续写小说
def qwen_completion_to_prompt(completion):
    return (
        f"<|im_start|>system\n"
        f"你是一个乐于助人的AI助手。请根据提供的内容回答问题。\n"
        f"<|im_end|>\n"
        f"<|im_start|>user\n"
        f"{completion}\n"
        f"<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

def qwen_messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        role = message.role
        content = message.content
        prompt += f"<|im_start|>{role}\n{content}\n<|im_end|>\n"
    prompt += "<|im_start|>assistant\n"
    return prompt

print("正在加载模型和分词器...")

# 加载 tokenizer 以便获取 eos_token_id
tokenizer = AutoTokenizer.from_pretrained(LOCAL_LLM_PATH, trust_remote_code=True)

llm = HuggingFaceLLM(
    model_name=LOCAL_LLM_PATH,
    tokenizer=tokenizer,
    context_window=8192,
    max_new_tokens=512,
    
    # 关键点 A: 告诉 LlamaIndex 怎么把文本转换成 Qwen 的聊天格式
    completion_to_prompt=qwen_completion_to_prompt,
    messages_to_prompt=qwen_messages_to_prompt,
    
    # 关键点 B: 生成参数控制
    generate_kwargs={
        "temperature": 0.1,         # RAG 任务建议低温度，减少胡言乱语
        "do_sample": True,
        "repetition_penalty": 1.1,  # 【重要】惩罚重复内容，设为 1.1 或 1.2
        # 【重要】Qwen 的特殊停止符，防止它停不下来
        # 151645 是 <|im_end|>, 151643 是 <|endoftext|>
        "eos_token_id": [151645, 151643, tokenizer.eos_token_id], 
    },
    device_map="auto",
)

Settings.llm = llm

正在加载模型和分词器...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# 2  chat

## 2.1 complete 模式



In [7]:
# 这种方法只适用于单次、无历史的提示
prompt = "写一个关于中国长城的小故事，不超过100字。"

# 使用 .complete() 方法
# 注意：通常 chat() 是推荐用于与 Chat 模型交互的方式，但 complete() 也可以用于简单的提示
try:
    response = Settings.llm.complete(prompt)
    print("--- LLM Response (Complete) ---")
    print(response.text)
    print("-------------------------------\n")
except AttributeError:
    print("Settings.llm 实例不支持 .complete() 方法，请使用包装后的 .chat()。")


--- LLM Response (Complete) ---
明朝边关，少年李山在长城上拾到一枚古铜铃。风起时，铃声悠悠，仿佛传来祖辈戍边的歌谣。他轻抚石壁，指尖触到斑驳刻痕——那是千年前守卫家国的誓言。长城不只砖石，是血脉相连的回响。
-------------------------------



## 2.2 chat 模式

### 2.2.1 一次

In [93]:
# 导入 ChatMessage，用于构造对话历史
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import Settings # 确保 Settings 在当前作用域可用

# 假设您想问一个关于中国的简单问题
prompt_content = "写一个关于中国长城的小故事，不超过100字。"

print(f"User Prompt: {prompt_content}\n")

# **关键修改：将字符串包装成 ChatMessage 列表**
messages = [
    ChatMessage(role=MessageRole.USER, content=prompt_content)
]

# 直接调用 llm 实例的 chat 方法，传入 messages 列表
response = Settings.llm.chat(messages)

print("--- LLM Response (Chat) ---")
print(response.message.content)
print("---------------------------\n")

User Prompt: 写一个关于中国长城的小故事，不超过100字。

--- LLM Response (Chat) ---
明朝边关，少年李远为守长城，日复一日巡山。风雪中，他发现敌军踪迹，奋勇出击，击退入侵者。夜深，他倚墙而立，望见月光洒在砖石上，仿佛听见祖先的低语——长城不只高耸，更在人心深处绵延不绝。
---------------------------



### 2.2.2 多次

In [94]:
from llama_index.core.chat_engine import SimpleChatEngine

# 使用 Settings 中配置的 llm 创建 ChatEngine
chat_engine = SimpleChatEngine.from_defaults(
    llm=Settings.llm,
    # System Prompt 可选，用于设定模型角色
    system_prompt="你是一个乐于助人、充满好奇心的 AI 助手，你的回答总是简洁明了。",
)

print("--- Chat Engine Conversation ---")

# 第一次提问
response_1 = chat_engine.chat("给我推荐一个适合在秋天阅读的中文小说名字。")
print(f"Assistant: {response_1.response}\n")

# 第二次提问（ChatEngine 会自动维护历史）
response_2 = chat_engine.chat("这个作者还有其他的作品吗？")
print(f"Assistant: {response_2.response}\n")

print("--------------------------------\n")

--- Chat Engine Conversation ---
Assistant: 《秋日童话》——温暖治愈，适合秋天阅读。

Assistant: 《秋日童话》的作者是张悦然，她还有其他优秀作品，比如：

- 《樱桃之远》  
- 《望月》  
- 《葵花》  

这些作品都以细腻的情感和诗意的语言著称，非常适合在秋天静心阅读。

--------------------------------



## 3 工具调用

In [9]:
from llama_index.core.memory import ChatMemoryBuffer 
import asyncio
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings 

# --- 1. 创建 Memory 对象 ---
# memory 是一个存储对话历史的缓冲区
# token_limit=3900 是一个示例值，应小于您的LLM上下文窗口 (例如 Qwen 7B 是 8192)
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

# ----------------- 定义工具 -----------------
def get_current_weather(city: str) -> str:
    """获取指定城市（例如：'北京'，'上海'）的当前天气信息。"""
    if "北京" in city:
        return "北京今天的气温是 28°C，多云转晴，空气质量良好。"
    elif "上海" in city:
        return "上海今天的气温是 -18°C，下雨，空气质量一般。"
    return f"抱歉，没有 {city} 的天气数据。"

def multiply(a:int, b:int ) -> int :
    """将两个数相乘(乘法运算)"""
    return a * b

weather_tool = FunctionTool.from_defaults(fn=get_current_weather)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
tools = [weather_tool, multiply_tool]
agent = ReActAgent(tools=tools, llm=Settings.llm, verbose=True,memory=memory)

In [10]:
import asyncio

async def run_agent_queries(agent_instance: ReActAgent):
    print("--- 调用 1: 工具函数 (乘法) ---")
    agent_query_calc = "调用乘法工具，计算一下 50 乘以 91 的结果是多少？"
    response_calc = await agent.run(user_msg="调用乘法工具，计算一下 50 乘以 91 的结果是多少？")
    print(f"最终回答: {response_calc}")

    print("--- 调用 2: 工具函数 (天气) ---")
    agent_query_weather = "告诉我上海今天的天气怎么样？"
    response_weather = await agent.run(user_msg=agent_query_weather)
    print(f"最终回答: {response_weather}")
    
await run_agent_queries(agent)

--- 调用 1: 工具函数 (乘法) ---
最终回答: 50 乘以 91 的结果是 4550。
--- 调用 2: 工具函数 (天气) ---
最终回答: 上海今天的气温是 -18°C，下雨，空气质量一般。


# Rag1

In [13]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings

documents = SimpleDirectoryReader("/root/Desktop/text").load_data()

index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine()

In [106]:
response = query_engine.query("孔乙己的过往人生经历是怎样的？")
print(response)

孔乙己的过往人生经历如下：

孔乙己原本读过书，曾有过一定的文化修养，但由于家庭贫困，最终未能通过科举考试进入仕途（“没有进学”）。他不会谋生技能，因此生活日渐艰难，逐渐陷入贫穷。为了维持生计，他曾靠为别人抄书换取饭食，但由于性格懒惰、贪图安逸，常常无法坚持工作，导致抄书的工作也难以持续，最终失去了这份收入来源。

由于经济状况恶化，他不得不偶尔从事偷窃行为来维持生活，尽管如此，在咸丰年间（文中未明确时间，但背景设定在晚清）的环境中，他仍保持着基本的道德操守，从不拖欠债务，即使一时无钱，也会在一个月内还清，因此在店里一直被记录在账上，名字也被擦去。

孔乙己一生孤傲，自视甚高，常引用儒家经典话语如“君子固穷”、“多乎哉？不多也”等，表现出知识分子的尊严与迂腐。然而，他的身份始终被他人轻视，人们嘲笑他“窃书不能算偷”，并以此作为讽刺和调侃的对象。

最终，孔乙己长期漂泊于社会边缘，逐渐消失在人们的视线中。文章结尾写道：“我到现在终于没有见——大约孔乙己的确死了。”反映出他对底层知识分子命运的悲悯与无奈，也揭示了当时社会冷漠、贫富悬殊以及人性冷漠的现实。


In [107]:
len(response.source_nodes)

2

In [108]:
response.source_nodes[0].text

'不一会，他喝完酒，便又在旁人的说笑声中，坐着用这手慢慢走去了。\n\n\u3000\u3000自此以后，又长久没有看见孔乙己。到了年关，掌柜取下粉板说，“孔乙己还欠十九个钱呢！”到第二年的端午，又说“孔乙己还欠十九个钱呢！”到中秋可是没有说，再到年关也没有看见他。\n\n\u3000\u3000我到现在终于没有见——大约孔乙己的确死了。\n\n\u3000\u3000一九一九年三月。⑻\n\n\u3000\u3000□注释\n\n\u3000\u3000⑴本篇最初发表于一九一九年四月《新青年》第六卷第四号。发表时篇末有作者的附记如下：“这一篇很拙的小说，还是去年冬天做成的。那时的意思，单在描写社会上的或一种生活，请读者看看，并没有别的深意。但用活字排印了发表，却已在这时候，——便是忽然有人用了小说盛行人身攻击的时候。大抵著者走入暗路，每每能引读者的思想跟他堕落：以为小说是一种泼秽水的器具，里面糟蹋的是谁。这实在是一件极可叹可怜的事。所以我在此声明，免得发生猜度，害了读者的人格。一九一九年三月二十六日记。”\n\n\u3000\u3000⑵描红纸：一种印有红色楷字，供儿童摹写毛笔字用的字帖。旧时最通行的一种，印有“上大人孔（明代以前作丘）乙己化三千七十士尔小生八九子佳作仁可知礼也”这样一些笔划简单、三字一句和似通非通的文字。\n\n\u3000\u3000⑶“君子固穷”：语见《论语·卫灵公》。“固穷”即“固守其穷”，不以穷困而改便操守的意思。\n\n\u3000\u3000⑷进学：明清科举制度，童生经过县考初试，府考复试，再参加由学政主持的院考（道考），考取的列名府、县学籍，叫进学，也就成了秀才。又规定每三年举行一次乡试（省一级考试），由秀才或监生应考，取中的就是举人。\n\n\u3000\u3000⑸回字有四样写法：回字通常只有三种写法：回、〔外“冂”内“巳”〕、〔“面”之下部〕。第四种写作〔外“囗”内“目”〕（见《康熙字典·备考》），极少见。\n\n\u3000\u3000⑹“多乎哉？不多也”：语见《论语·子罕》：“大宰问于子贡曰：‘夫子圣者与？何其多能也！’子贡曰：‘固天纵之将圣，又多能也。’子闻之，曰：‘大宰知我乎？吾少也贱，故多能鄙事。’君子多乎哉？不多也。'

In [109]:
response.source_nodes[1].text

'因为他姓孔，别人便从描红纸⑵上的“上大人孔乙己”这半懂不懂的话里，替他取下一个绰号，叫作孔乙己。孔乙己一到店，所有喝酒的人便都看着他笑，有的叫道，“孔乙己，你脸上又添上新伤疤了！”他不回答，对柜里说，“温两碗酒，要一碟茴香豆。”便排出九文大钱。他们又故意的高声嚷道，“你一定又偷了人家的东西了！”孔乙己睁大眼睛说，“你怎么这样凭空污人清白……”“什么清白？我前天亲眼见你偷了何家的书，吊着打。”孔乙己便涨红了脸，额上的青筋条条绽出，争辩道，“窃书不能算偷……窃书！……读书人的事，能算偷么？”接连便是难懂的话，什么“君子固穷”⑶，什么“者乎”之类，引得众人都哄笑起来：店内外充满了快活的空气。\n\n\u3000\u3000听人家背地里谈论，孔乙己原来也读过书，但终于没有进学⑷，又不会营生；于是愈过愈穷，弄到将要讨饭了。幸而写得一笔好字，便替人家钞钞书，换一碗饭吃。可惜他又有一样坏脾气，便是好吃懒做。坐不到几天，便连人和书籍纸张笔砚，一齐失踪。如是几次，叫他钞书的人也没有了。孔乙己没有法，便免不了偶然做些偷窃的事。但他在我们店里，品行却比别人都好，就是从不拖欠；虽然间或没有现钱，暂时记在粉板上，但不出一月，定然还清，从粉板上拭去了孔乙己的名字。\n\n\u3000\u3000孔乙己喝过半碗酒，涨红的脸色渐渐复了原，旁人便又问道，“孔乙己，你当真认识字么？”孔乙己看着问他的人，显出不屑置辩的神气。他们便接着说道，“你怎的连半个秀才也捞不到呢？”孔乙己立刻显出颓唐不安模样，脸上笼上了一层灰色，嘴里说些话；这回可是全是之乎者也之类，一些不懂了。在这时候，众人也都哄笑起来：店内外充满了快活的空气。\n\n\u3000\u3000在这些时候，我可以附和着笑，掌柜是决不责备的。而且掌柜见了孔乙己，也每每这样问他，引人发笑。孔乙己自己知道不能和他们谈天，便只好向孩子说话。有一回对我说道，“你读过书么？'

In [14]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\n only Using"
    " the context information, not using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you will not "
    " answer the question on your own.\n 所有回答以中文显示 \n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing the new"
    " context , update or repeat the existing answer.\n 所有回答以中文显示 \n"
)
refine_template = PromptTemplate(refine_template_str)

In [15]:
res=index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
    ).query("孔乙己是用什么手势付钱的？")
print(res.response)

孔乙己是用一只手（满手是泥）付钱的手势付钱的。文中提到：“见他满手是泥，原来他便用这手走来的。” 这说明他用手走路并用这只手付钱。
